In [ ]:
# installing kaggle library
!pip install kaggle

Upload your kaggle.json file

In [ ]:
# configure the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.12GB/s]


In [ ]:
# extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing

In [ ]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
# checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [ ]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns and reading the datasets again
column_names = ['target', 'id', 'data', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding='ISO-8859-1')

In [ ]:
# checking the number of rows and columns
twitter_data.shape

(1600000, 6)

In [ ]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,target,id,data,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
data,0
flag,0
user,0
text,0


In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Convert the target "4" to "1"

In [ ]:
twitter_data['target'] = twitter_data['target'].replace(4,1)

In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 --> Negative Tweet

1 --> Positive Tweet

**Stemming**

Stemming is a process of reducing a word to it's Root word

example: actor, actress, acting = act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

Downsample

In [ ]:
# random downsample
twitter_data = twitter_data.sample(n=10000)

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,data,flag,user,text,stemmed_content
985793,1,1834508742,Mon May 18 03:12:15 PDT 2009,NO_QUERY,electricfm,It's back to work! We'll get you through the M...,back work get monday blue
398265,0,2056841612,Sat Jun 06 11:53:48 PDT 2009,NO_QUERY,RyanPerezTHAN,im bored at work. I don't want to work tonight,im bore work want work tonight
1312948,1,2013719236,Tue Jun 02 23:23:30 PDT 2009,NO_QUERY,gz_babii,Confession: Ive seen the movie &quot;Selena&qu...,confess ive seen movi quot selena quot like x ...
824115,1,1556166560,Sat Apr 18 21:51:58 PDT 2009,NO_QUERY,tracyr0bles,@trishazurin So let's get on with the gossip ...,trishazurin let get gossip laaav gossip
1338811,1,2018353718,Wed Jun 03 09:50:11 PDT 2009,NO_QUERY,Laphroaigwhisky,@HeatherRETax you cant be to careful and aft...,heatherretax cant care long lunch snooz time f...


In [ ]:
print(twitter_data['stemmed_content'])

985793                             back work get monday blue
398265                        im bore work want work tonight
1312948    confess ive seen movi quot selena quot like x ...
824115               trishazurin let get gossip laaav gossip
1338811    heatherretax cant care long lunch snooz time f...
                                 ...                        
1429617    shall call squishi shall mine shall squishi fi...
915919                  total love lip ring pierc ut favorit
1089313                        mum til monday morn say hello
1515375                    jamesmoor well that good think et
1169488                             phlaymzgudmoney yep sure
Name: stemmed_content, Length: 10000, dtype: object


In [ ]:
print(twitter_data['target'])

985793     1
398265     0
1312948    1
824115     1
1338811    1
          ..
1429617    1
915919     1
1089313    1
1515375    1
1169488    1
Name: target, Length: 10000, dtype: int64


In [ ]:
# separating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['back work get monday blue' 'im bore work want work tonight'
 'confess ive seen movi quot selena quot like x thatz shi' ...
 'mum til monday morn say hello' 'jamesmoor well that good think et'
 'phlaymzgudmoney yep sure']


In [ ]:
print(Y)

[1 0 1 ... 1 1 1]


Splitting the data into training data and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(10000,) (8000,) (2000,)


In [ ]:
print(X_test)

['hate tire want sleep' 'edwardslinjail still freakin twitter jail hour'
 'camil nicol luv song' ... 'pamdemonium time'
 'http twitpic com dc b new haircut' 'feel guilti happi fuck']


In [ ]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 59324 stored elements and shape (8000, 12973)>
  Coords	Values
  (0, 753)	0.3286765974928104
  (0, 1693)	0.21661310754954277
  (0, 3628)	0.3286765974928104
  (0, 4899)	0.2705510916814101
  (0, 5177)	0.30561686983746755
  (0, 8693)	0.2882831269250422
  (0, 9253)	0.21572144512978036
  (0, 9500)	0.19794935734805294
  (0, 9938)	0.17813374624145628
  (0, 10662)	0.36462401854165544
  (0, 11400)	0.26108995266097007
  (0, 12237)	0.18919511056328014
  (0, 12561)	0.36462401854165544
  (1, 3559)	1.0
  (2, 381)	0.25922265828232904
  (2, 578)	0.41467701473311297
  (2, 978)	0.43359281574464453
  (2, 1052)	0.3585091484124191
  (2, 4306)	0.43359281574464453
  (2, 4347)	0.17328667108297552
  (2, 6749)	0.1909432881080037
  (2, 11462)	0.36342443830741084
  (2, 12312)	0.24030685727079745
  (3, 4976)	0.5773502691896257
  (3, 6995)	0.5773502691896257
  :	:
  (7996, 11369)	0.31748006815566365
  (7997, 203)	0.2406885216002241
  (7997, 1693)	0.20474

In [ ]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 12436 stored elements and shape (2000, 12973)>
  Coords	Values
  (0, 4809)	0.5187433300037532
  (0, 10316)	0.49311343885033926
  (0, 11466)	0.5486545337694366
  (0, 12266)	0.4321142170492105
  (1, 4137)	0.5517777574114493
  (1, 5111)	0.3558057135534658
  (1, 5592)	0.612125794120971
  (1, 10732)	0.3043675289439001
  (1, 11799)	0.31875699227633514
  (2, 6819)	0.5803030194107109
  (2, 7937)	0.6875312633914893
  (2, 10461)	0.43651937817480047
  (3, 1243)	0.23408152513062014
  (3, 5101)	0.8730432611120559
  (3, 5151)	0.2587459477086021
  (3, 6543)	0.11788726049014858
  (3, 8510)	0.24387606589104624
  (3, 12350)	0.20658871378504903
  (4, 2700)	0.223069493456975
  (4, 4347)	0.22204566125029535
  (4, 4679)	0.3229471436499617
  (4, 5312)	0.26672589744649616
  (4, 7311)	0.35430267279093125
  (4, 9852)	0.3205556662041904
  (4, 9860)	0.48992222385944034
  :	:
  (1995, 4490)	0.27178300556509605
  (1995, 5140)	0.2864546779279877
  (1995, 

Training the Machine Learning Model

Logistic Regression

In [ ]:
model = LogisticRegression(max_iter=100)
model.fit(X_train, Y_train)

LogisticRegression()

Model Evaluation

Accuracy Score

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy on the training data: ', training_data_accuracy)

Accuracy on the training data:  0.866125


In [ ]:
X_test_prediction = model.predict(X_test)
training_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy on the test data: ', training_data_accuracy)

Accuracy on the test data:  0.7215


Saving the trained model

In [ ]:
filename = 'trained_model.sav'

In [ ]:
import pickle

Using the saved model for future predictions

In [ ]:
pickle.dump(model, open(filename, 'wb'))

Using the saved model for future predictions

In [ ]:
# Loading the saved model
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
X_new = X_test[3]
print(Y_test[3])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

1
[0]
Negative Tweet
